# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [76]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [77]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [78]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [79]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [80]:
# Create a connection to the local db (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [81]:
# Create a new keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [82]:
# Set the working keyspace
try:
    session.set_keyspace('sparkify_udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [67]:
# Drop the music_app_history_by_session table if it exists then create it
try:
    session.execute("""DROP TABLE IF EXISTS music_app_history_by_session"""
)

except Exception as e:
    print(e) 

# Table designed to meet the following requirement    
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182    
# partitioned by sessionid,  including clustering key of iteminsession in the primary key
    
try:
    session.execute("""CREATE TABLE IF NOT EXISTS music_app_history_by_session (
                        sessionid INT,
                        iteminsession INT,
                        artist TEXT,
                        song TEXT,
                        length FLOAT,
                        PRIMARY KEY (sessionId, iteminsession));"""
)

except Exception as e:
    print(e) 

                    

In [68]:
# read the csv file into the music_app_history_by_session table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history_by_session (sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [69]:
# Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "select artist, song, length from music_app_history_by_session where sessionid = 338 and iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [70]:
# Drop the music_app_history_by_session_and_user table if it exists then create it
try:
    session.execute("""DROP TABLE IF EXISTS music_app_history_by_session_and_user"""
)

except Exception as e:
    print(e) 

# Table designed to meet the following requirement    
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182    
# partitioned by userid, including clustering keys of sessionid and iteminsession in the primary key to give uniqueness and order

try:
    session.execute("""CREATE TABLE IF NOT EXISTS music_app_history_by_session_and_user (
                        userid INT,
                        sessionid INT,
                        iteminsession INT,
                        artist TEXT,
                        song TEXT,
                        firstname TEXT,
                        lastname TEXT,
                        PRIMARY KEY (userid, sessionid, iteminsession));"""
)

except Exception as e:
    print(e)          
       

In [71]:
# read the csv file into the music_app_history_by_session_and_user table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history_by_session_and_user (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [72]:
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "select * from music_app_history_by_session_and_user where userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [73]:
# Drop the music_app_history_by_song table if it exists then create it
try:
    session.execute("""DROP TABLE IF EXISTS music_app_history_by_song"""
)

except Exception as e:
    print(e) 

# Table designed to meet the following requirement    
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'  
# partitioned by songid, including clustering keys of sessionid and iteminsession in the primary key to give uniqueness

try:
    session.execute("""CREATE TABLE IF NOT EXISTS music_app_history_by_song (
                        song TEXT,
                        sessionid INT,
                        iteminsession INT,
                        firstname TEXT,
                        lastname TEXT,
                        PRIMARY KEY (song, sessionid, iteminsession));"""
)

except Exception as e:
    print(e)

In [74]:
# read the csv file into the music_app_history_by_song table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history_by_song (song, sessionid, iteminsession, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[8]), int(line[3]), line[1], line[4]))

In [75]:
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select firstname, lastname from music_app_history_by_song where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [57]:
## TO-DO: Drop the table before closing out the sessions

In [58]:
try:
    session.execute("""DROP TABLE IF EXISTS music_app_history_by_session"""
)

except Exception as e:
    print(e) 
try:
    session.execute("""DROP TABLE IF EXISTS music_app_history_by_session_and_user"""
)

except Exception as e:
    print(e) 
try:
    session.execute("""DROP TABLE IF EXISTS music_app_history_by_song"""
)

except Exception as e:
    print(e) 

### Close the session and cluster connection¶

In [59]:
session.shutdown()
cluster.shutdown()